In [1]:
# cell 1: Configuration and Library Imports
# IMPORTANT: Run this cell first!

# Jupyter magic command to enable Tkinter integration
# This allows the Tkinter GUI to run interactively without blocking the notebook.
%gui tk

import os
import tkinter as tk
from tkinter import scrolledtext, messagebox, filedialog, Label
from dotenv import load_dotenv
import threading
import queue
from PIL import Image, ImageTk # Pillow for image handling, ImageTk for Tkinter display
import datetime

# --- Configuration ---
# Load environment variables (e.g., OPENAI_API_KEY if using OpenAI fallback)
load_dotenv()

# --- Tkinter GUI Styling Variables ---
# Choose a sleek, modern dark theme
BG_COLOR = "#2b2b2b"      # Dark background
FG_COLOR = "#e0e0e0"      # Light foreground text
INPUT_BG = "#3c3c3c"      # Slightly lighter background for input fields
BUTTON_COLOR = "#4CAF50"  # Green for main buttons
ACCENT_COLOR = "#2196F3"  # Blue for accents/other buttons
BORDER_COLOR = "#505050"  # Border for frames/widgets
FONT_FAMILY = "Inter"     # Modern font
FONT_SIZE_NORMAL = 10
FONT_SIZE_LARGE = 12
FONT_SIZE_SMALL = 8


# --- Try to Import Core AI Image Generation Libraries ---
# This block attempts to import necessary libraries.
# If any are missing, it will show a user-friendly error message.
try:
    import torch
    from diffusers import DiffusionPipeline
    # Attempt to install specific Hugging Face components if not already
    try:
        import huggingface_hub
        import tqdm
    except ImportError:
        # Fallback if hf_xet or tqdm aren't found initially
        # Use os.system for !pip commands within Jupyter context if needed,
        # but standard pip install is usually better outside the initial run
        print("Installing huggingface_hub[hf_xet] and tqdm for better download feedback...")
        os.system("pip install huggingface_hub[hf_xet] tqdm")
        import huggingface_hub
        import tqdm

except ImportError as e:
    # If core AI libraries are missing, show a message and exit
    messagebox.showerror("Dependencies Missing",
                         f"A critical AI library is missing: {e}\n\n"
                         "Please ensure you have run ALL 'pip install' commands in a new terminal:\n\n"
                         "pip install torch torchvision torchaudio\n"
                         "pip install diffusers transformers accelerate\n"
                         "pip install Pillow ipywidgets\n"
                         "pip install huggingface_hub[hf_xet] tqdm")
    # In Jupyter, exiting might just stop the cell. User needs to restart kernel if severe.
    raise SystemExit("Missing critical dependencies. Please install them and restart Jupyter Kernel.")


print("Cell 1: Configuration and initial imports complete. Now run Cell 2 to define classes.")

Cell 1: Configuration and initial imports complete. Now run Cell 2 to define classes.


In [2]:
# cell 2: AI Backend and GUI Classes Definition
# Run this cell after Cell 1.

# --- Backend Logic for AI Image Generation ---
class ImageGeneratorBackend:
    """
    Handles the core logic for AI image generation, including device configuration,
    model loading, and the actual image generation process.
    """
    def __init__(self, model_id="stabilityai/stable-diffusion-xl-base-1.0"):
        self.model_id = model_id
        self.pipe = None
        self.device = self._configure_device()

        # Try to load the model immediately upon initialization
        self.load_model()

    def _configure_device(self):
        """
        Determines the best device (MPS for Apple Silicon, otherwise CPU) for PyTorch.
        """
        if torch.backends.mps.is_available():
            print("Backend: Using Apple Silicon MPS backend for GPU acceleration.")
            return "mps"
        else:
            print("Backend: Apple Silicon MPS backend not available. Falling back to CPU. This will be very slow.")
            return "cpu"

    def load_model(self):
        """
        Loads the Stable Diffusion model. This is a heavy operation and should be done once.
        """
        print(f"Backend: Loading Stable Diffusion model: {self.model_id}. This is a one-time download (~5-10GB).")
        print("Backend: You might see 'Error displaying widget' in Jupyter; this means the visual progress bar isn't rendering, but the download is still active. Check your terminal where Jupyter was launched for text-based progress updates.")
        try:
            # Use torch_dtype=torch.float16 for memory efficiency on MPS
            self.pipe = DiffusionPipeline.from_pretrained(self.model_id, torch_dtype=torch.float16, use_safetensors=True)
            self.pipe = self.pipe.to(self.device)
            print("Backend: Stable Diffusion model loaded successfully.")
            return True
        except Exception as e:
            print(f"Backend: Error loading model: {e}")
            self.pipe = None
            return False

    def generate_image(self, prompt, num_inference_steps=30, guidance_scale=4):
        """
        Generates an image based on the given prompt and parameters.
        Returns the PIL Image object.
        """
        if self.pipe is None:
            raise ValueError("AI Model not loaded. Cannot generate image.")

        print(f"Backend: Generating image for prompt: '{prompt}'...")
        try:
            generated_image = self.pipe(
                prompt=prompt,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale
            ).images[0]
            print("Backend: Image generation complete.")
            return generated_image
        except Exception as e:
            print(f"Backend: Error during image generation: {e}")
            raise # Re-raise to be caught by the GUI's error handling


# --- Tkinter GUI Application ---
class ImageGeneratorGUI:
    """
    Creates and manages the Tkinter-based graphical user interface for the image generator.
    """
    def __init__(self, master, backend):
        self.master = master
        self.backend = backend
        self.master.title("AI Image Generator")
        self.master.geometry("1000x800") # Initial window size
        self.master.minsize(700, 600)    # Minimum window size
        self.master.configure(bg=BG_COLOR)

        # Queue for thread-safe communication
        self.image_queue = queue.Queue()

        self._configure_styles()
        self._create_widgets()
        self._initial_status_check()

        # Start periodic check for results from the worker thread
        self.master.after(100, self._check_queue)
        
        # Hide the root window initially, then show it once everything is set up.
        # This helps prevent flashing of an unstyled window.
        self.master.withdraw()
        self.master.after(0, self.master.deiconify)


    def _configure_styles(self):
        """Applies consistent styling to Tkinter widgets."""
        self.master.option_add('*Font', f'{FONT_FAMILY} {FONT_SIZE_NORMAL}')
        self.master.option_add('*Background', BG_COLOR)
        self.master.option_add('*Foreground', FG_COLOR)

        # Style for Entry widgets
        self.master.option_add('*TEntry.background', INPUT_BG)
        self.master.option_add('*TEntry.foreground', FG_COLOR)
        self.master.option_add('*TEntry.fieldbackground', INPUT_BG)
        self.master.option_add('*TEntry.bordercolor', BORDER_COLOR)
        self.master.option_add('*TEntry.relief', 'solid')
        self.master.option_add('*TEntry.borderwidth', 1)
        self.master.option_add('*TEntry.insertBackground', FG_COLOR) # Blinking cursor color

        # Style for Button widgets
        self.master.option_add('*TButton.background', BUTTON_COLOR)
        self.master.option_add('*TButton.foreground', "white")
        self.master.option_add('*TButton.focuscolor', ACCENT_COLOR)
        self.master.option_add('*TButton.relief', 'flat')
        self.master.option_add('*TButton.borderwidth', 0)
        self.master.option_add('*TButton.font', f'{FONT_FAMILY} {FONT_SIZE_NORMAL} bold')
        self.master.option_add('*TButton.activebackground', '#388E3C') # Darker green on click
        self.master.option_add('*TButton.activeforeground', 'white')

        # Style for Label widgets (if using ttk.Label)
        self.master.option_add('*TLabel.background', BG_COLOR)
        self.master.option_add('*TLabel.foreground', FG_COLOR)

        # Configure columns and rows to be responsive
        self.master.grid_columnconfigure(0, weight=1)
        self.master.grid_rowconfigure(0, weight=0) # Controls frame - fixed size
        self.master.grid_rowconfigure(1, weight=1) # Image display - takes available space
        self.master.grid_rowconfigure(2, weight=0) # Status label - fixed size


    def _create_widgets(self):
        """Creates all the Tkinter widgets for the GUI layout."""
        # --- Controls Frame (Top) ---
        controls_frame = tk.Frame(self.master, bg=BG_COLOR, bd=2, relief="solid", highlightbackground=BORDER_COLOR, highlightthickness=1)
        controls_frame.grid(row=0, column=0, padx=10, pady=10, sticky="ew")
        controls_frame.grid_columnconfigure(0, weight=1) # Prompt input takes most space
        controls_frame.grid_columnconfigure(1, weight=0) # Button fixed size

        prompt_label = tk.Label(controls_frame, text="Enter your image prompt:", bg=BG_COLOR, fg=FG_COLOR, font=(FONT_FAMILY, FONT_SIZE_NORMAL, 'bold'))
        prompt_label.grid(row=0, column=0, columnspan=2, padx=10, pady=(10, 5), sticky="w")

        self.prompt_entry = tk.Entry(controls_frame, width=80, bg=INPUT_BG, fg=FG_COLOR, insertbackground=FG_COLOR,
                                     font=(FONT_FAMILY, FONT_SIZE_LARGE))
        self.prompt_entry.grid(row=1, column=0, padx=10, pady=(0, 10), sticky="ew")
        self.prompt_entry.bind("<Return>", self._trigger_generation_on_enter) # Bind Enter key

        self.generate_button = tk.Button(controls_frame, text="Generate Image", command=self._start_generation_thread,
                                        cursor="hand2", bg=BUTTON_COLOR, fg="Black", activebackground='#388E3C')
        self.generate_button.grid(row=1, column=1, padx=(0, 10), pady=(0, 10), sticky="e")

        # --- Image Display Frame (Middle) ---
        image_frame = tk.Frame(self.master, bg=BG_COLOR, bd=2, relief="solid", highlightbackground=BORDER_COLOR, highlightthickness=1)
        image_frame.grid(row=1, column=0, padx=10, pady=0, sticky="nsew")
        image_frame.grid_rowconfigure(0, weight=1)
        image_frame.grid_columnconfigure(0, weight=1)

        # Placeholder for the image
        self.image_label = Label(image_frame, bg=INPUT_BG, text="Generated image will appear here", fg="#888888",
                                 font=(FONT_FAMILY, FONT_SIZE_LARGE))
        self.image_label.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)
        
        # Keep a reference to the image to prevent it from being garbage collected
        self.current_tk_image = None 

        # --- Status Bar (Bottom) ---
        self.status_label = tk.Label(self.master, text="Waiting for prompt...", bg=ACCENT_COLOR, fg="white",
                                     font=(FONT_FAMILY, FONT_SIZE_NORMAL, 'bold'), anchor="w", padx=10, pady=5)
        self.status_label.grid(row=2, column=0, padx=10, pady=10, sticky="ew")

    def _initial_status_check(self):
        """Checks if the model loaded successfully and updates status."""
        if self.backend.pipe is None:
            self.status_label.config(text="Error: Model failed to load! Check terminal output.", bg="#FF6347") # Tomato red for error
            self.generate_button.config(state=tk.DISABLED) # Disable button on error
        else:
            self.status_label.config(text="Model ready. Enter your prompt and click 'Generate Image'.", bg=BUTTON_COLOR)

    def _trigger_generation_on_enter(self, event=None):
        """Handles pressing Enter in the prompt entry to trigger generation."""
        self._start_generation_thread()

    def _start_generation_thread(self):
        """
        Starts a new thread to generate the image, keeping the GUI responsive.
        """
        prompt = self.prompt_entry.get().strip()

        if not prompt:
            messagebox.showwarning("Empty Prompt", "Please enter a text prompt to generate an image.")
            return

        self.status_label.config(text="Generating image... This may take a while.", bg=ACCENT_COLOR)
        self.generate_button.config(state=tk.DISABLED) # Disable button during generation
        self.prompt_entry.config(state=tk.DISABLED) # Disable input during generation

        # Start the generation in a separate thread
        threading.Thread(target=self._run_generation_in_thread, args=(prompt,)).start()

    def _run_generation_in_thread(self, prompt):
        """
        Executes the image generation in a separate thread and puts the result in a queue.
        """
        try:
            generated_image = self.backend.generate_image(prompt)
            self.image_queue.put({"status": "success", "image": generated_image, "prompt": prompt})
        except Exception as e:
            self.image_queue.put({"status": "error", "message": str(e)})

    def _check_queue(self):
        """
        Periodically checks the queue for results from the worker thread.
        Updates the GUI with the results.
        """
        try:
            result = self.image_queue.get(block=False) # Non-blocking get

            self.generate_button.config(state=tk.NORMAL) # Re-enable button
            self.prompt_entry.config(state=tk.NORMAL)    # Re-enable input
            self.prompt_entry.focus_set()                # Focus back to prompt

            if result["status"] == "success":
                generated_image = result["image"]
                prompt_text = result["prompt"]

                # Generate unique filename with timestamp and cleaned prompt
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                clean_prompt_part = "".join(c for c in prompt_text if c.isalnum() or c in (' ', '.', '_')).strip()
                if len(clean_prompt_part) > 50:
                    clean_prompt_part = clean_prompt_part[:50].strip() + "..."
                
                # Replace potentially invalid characters for filenames
                file_name_suffix = clean_prompt_part.replace(" ", "_").replace("...", "")
                file_name_suffix = "".join(c if c.isalnum() or c == '_' else '' for c in file_name_suffix) # Final cleanup
                if not file_name_suffix: # Fallback if prompt becomes empty after cleaning
                    file_name_suffix = "untitled"

                output_dir = "generated_images"
                os.makedirs(output_dir, exist_ok=True)
                file_path = os.path.join(output_dir, f"image_{timestamp}_{file_name_suffix}.png")
                
                generated_image.save(file_path)
                self.status_label.config(text=f"Image saved to: {file_path}", bg=BUTTON_COLOR)

                # Resize image for display in GUI while maintaining aspect ratio
                # Get current size of the label where image will be displayed
                # Use master.update_idletasks() to ensure widget geometry is up-to-date
                self.master.update_idletasks()
                label_width = self.image_label.winfo_width() - 20 # Adjust for padding
                label_height = self.image_label.winfo_height() - 20

                if label_width > 0 and label_height > 0:
                    original_width, original_height = generated_image.size
                    ratio = min(label_width / original_width, label_height / original_height)
                    new_width = int(original_width * ratio)
                    new_height = int(original_height * ratio)
                    resized_image = generated_image.resize((new_width, new_height), Image.LANCZOS)
                else:
                    # If label hasn't rendered yet (e.g., very first time), use a default size
                    resized_image = generated_image.resize((512, 512), Image.LANCZOS) # Default display size


                self.current_tk_image = ImageTk.PhotoImage(resized_image) # Keep reference!
                self.image_label.config(image=self.current_tk_image, text="")
                self.image_label.image = self.current_tk_image # Store reference directly on the label

            elif result["status"] == "error":
                self.status_label.config(text=f"Error: {result['message']}", bg="#FF6347") # Error color
                messagebox.showerror("Image Generation Error", result['message'])

        except queue.Empty:
            pass # No results yet, check again later

        # Schedule the next check (important to keep checking for results)
        self.master.after(100, self._check_queue)

print("Cell 2: AI Backend and GUI classes defined. Now run Cell 3 to initialize and launch the GUI.")


Cell 2: AI Backend and GUI classes defined. Now run Cell 3 to initialize and launch the GUI.


In [3]:
# cell 3: Initialize and Launch the GUI
# Run this cell after Cell 1 and Cell 2.
# This cell will display the Tkinter window.

print("Cell 3: Starting AI Image Generator setup...")
backend = None
try:
    backend = ImageGeneratorBackend()
    if backend.pipe is None:
        # If model loading failed inside backend, it's already reported.
        # We exit here if the backend couldn't initialize its core component.
        print("Failed to initialize backend model. Please check the terminal output from Cell 2 for errors.")
        messagebox.showerror("Initialization Error", "AI model failed to load. Please check your internet connection, disk space, and ensure your PyTorch installation supports MPS.")
    else:
        root = tk.Tk()
        app = ImageGeneratorGUI(root, backend)
        print("Launching GUI...")
        # root.mainloop() is handled by %gui tk magic command
        # The window will appear once this cell finishes executing.
        
except Exception as e:
    messagebox.showerror("Initialization Error", f"Failed to initialize application: {e}\n\n"
                                                  "Please ensure all dependencies are installed, "
                                                  "you have enough RAM, and a stable internet connection.")
    print(f"Error during GUI initialization: {e}")

print("Cell 3: GUI launch sequence completed. The image generator window should now be visible.")
print("If you closed the GUI window, you might need to restart your Jupyter kernel and run all cells again to relaunch it.")

Cell 3: Starting AI Image Generator setup...
Backend: Using Apple Silicon MPS backend for GPU acceleration.
Backend: Loading Stable Diffusion model: stabilityai/stable-diffusion-xl-base-1.0. This is a one-time download (~5-10GB).
Backend: You might see 'Error displaying widget' in Jupyter; this means the visual progress bar isn't rendering, but the download is still active. Check your terminal where Jupyter was launched for text-based progress updates.


Couldn't connect to the Hub: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models/stabilityai/stable-diffusion-xl-base-1.0 (Request ID: Root=1-685cf58c-6306404d32b01b5b404f10ab;c4d0dbfa-e570-4c8a-8e18-23872615cb6e)

Invalid credentials in Authorization header.
Will try to load from local cache.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Backend: Stable Diffusion model loaded successfully.
Launching GUI...
Cell 3: GUI launch sequence completed. The image generator window should now be visible.
If you closed the GUI window, you might need to restart your Jupyter kernel and run all cells again to relaunch it.
